In [271]:
import cv2
import numpy as np

In [272]:
def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

In [273]:
def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    c1 = x , y
    c2 = x_plus_w, y_plus_h
    label = str(classes[class_id])
    color = COLORS[class_id]
    cv2.rectangle(img, c1, c2,color, 1)
    t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
    c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
    cv2.rectangle(img, c1, c2,color, -1)
    cv2.putText(img, label, (c1[0], c1[1] + t_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1, [225,255,255], 1)
   

In [274]:
image = cv2.imread("yolo_images/flower.jpeg")


In [275]:
Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392

In [276]:
classes = None

In [277]:
with open("yolov3.txt", 'r') as f:
    classes = [line.strip() for line in f.readlines()]

In [278]:
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

In [279]:
weights = "yolov3.weights"
config = "yolov3.cfg"

net = cv2.dnn.readNet(weights,config)

In [280]:
blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
net.setInput(blob)
outs = net.forward(get_output_layers(net))

In [281]:
class_ids = []
confidences = []
boxes = []
conf_threshold = 0.5
nms_threshold = 0.4

In [282]:
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])

In [283]:
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

In [284]:
for i in indices:
    i = i[0]
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    draw_prediction(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

In [285]:
cv2.imshow("object detection", image)
cv2.waitKey()
    
cv2.imwrite("yolo_images/object-detection.jpg", image)
cv2.destroyAllWindows()